# QuCoNot Module Guide

In this Jupyter notebook, we're going to demonstrate the multi-controlled Toffoli implementations, verifications and transformations based on the manuscript "Classification of permutation implementations for quantum computing".

In order to run the script in this notebook, a `qiskit-aer` package is required which is not a primary dependency of the package. You can install it by running the following block. 

In [1]:
%pip install qiskit-aer

Note: you may need to restart the kernel to use updated packages.


## Implementations

We have implemented some of the multi-controlled Toffoli implementations listed in Table 1. First, we will start by importing each implementation.

<!-- <div>
<img src="img/summary_mct.png" width="800"/>
</div> -->

In [2]:
from qiskit import transpile
from quconot.implementations import (
    MCTBarenco74Dirty, 
    MCTBarenco75Dirty,
    MCTCleanWastingEntangling,
    MCTDirtyWastingEntangling, 
    MCTNoAuxiliary, 
    MCTNoAuxiliaryRelative,
    MCTParallelDecomposition,
    MCTRecursion,
    MCTVChain,
    MCTVChainDirty,
    MCTQclibLdmcu
)

## Verifications

Next, we import the functions needed for verifying the implementations. Verification for each implementation is summarized in Table 2 from the paper.

<!-- <div>
<img src="img/verification_table.png" width="800"/>
</div> -->

In [3]:
from quconot.verifications import (
    verify_circuit_strict_clean_non_wasting,
    verify_circuit_relative_clean_non_wasting,
    verify_circuit_strict_clean_wasting_entangled,
    verify_circuit_relative_clean_wasting_separable,
    verify_circuit_strict_clean_wasting_separable,
    verify_circuit_strict_dirty_non_wasting,
    verify_circuit_relative_dirty_non_wasting,
    verify_circuit_strict_dirty_wasting_entangled,
    verify_circuit_relative_dirty_wasting_separable,
    verify_circuit_strict_dirty_wasting_separable
)

Using the imported functions, we create the necessary functions for printing the verification result.

In [4]:
from qiskit import Aer

usim = Aer.get_backend('unitary_simulator')
    

def get_ref_unitary(control_no):
    mct = MCTNoAuxiliary(control_no)
    circ = mct.generate_circuit()
    circ = transpile(circ, basis_gates=["cx", "u3"])
    return usim.run(circ).result().get_unitary()
        
def verify_all(tested_matrix, ref_unitary):
    rd = {}

    rd["SCNW"] = verify_circuit_strict_clean_non_wasting(tested_matrix, ref_unitary)
    rd["RCNW"] = verify_circuit_relative_clean_non_wasting(tested_matrix, ref_unitary)
    rd["SCWE"] = verify_circuit_strict_clean_wasting_entangled(tested_matrix, ref_unitary)
    rd["RCWS"] = verify_circuit_relative_clean_wasting_separable(tested_matrix, ref_unitary)
    rd["SCWS"] = verify_circuit_strict_clean_wasting_separable(tested_matrix, ref_unitary)
    rd["SDNW"] = verify_circuit_strict_dirty_non_wasting(tested_matrix, ref_unitary)
    rd["RDNW"] = verify_circuit_relative_dirty_non_wasting(tested_matrix, ref_unitary)
    rd["SDWE"] = verify_circuit_strict_dirty_wasting_entangled(tested_matrix, ref_unitary)
    rd["RDWS"] = verify_circuit_relative_dirty_wasting_separable(tested_matrix, ref_unitary)
    rd["SDWS"] = verify_circuit_strict_dirty_wasting_separable(tested_matrix, ref_unitary)
    
    for k in rd:
        if rd[k][0]:
            print("This implementation belongs to " + k)
        else:
            print("This implementation doesn't belong to " + k)
    

## Application

For a particular implementation, we will first generate the quantum circuit for MCT with 5 control qubits. Next, we will get the corresponding unitary. Finally, we will verify whether each implementation is correct using the imported functions. Note that the verifications should also follow the inclusion relations illustrated in Figure 1. 

<div>
<img src="img/classification.png" width="500"/>
</div>


### Barenco

Now we have an example for Barenco 74, which is considered to be in the class "Strict Dirty Non-Wasting". Based on the above DAG, it should pass all the verifications associated with any class that is a superset of S-D-NW.

In [6]:
control_no = 5
mct = MCTBarenco74Dirty(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation belongs to SCNW
This implementation belongs to RCNW
This implementation belongs to SCWE
This implementation belongs to RCWS
This implementation belongs to SCWS
This implementation belongs to SDNW
This implementation belongs to RDNW
This implementation belongs to SDWE
This implementation belongs to RDWS
This implementation belongs to SDWS


As you can see from the result, the clean and dirty auxiliary tests are failed because they are outside of the DAG path.

Now, we have Barenco 75 which is an implementation without auxiliary:

In [7]:
control_no = 5
mct = MCTBarenco75Dirty(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation belongs to SCNW
This implementation belongs to RCNW
This implementation belongs to SCWE
This implementation belongs to RCWS
This implementation belongs to SCWS
This implementation belongs to SDNW
This implementation belongs to RDNW
This implementation belongs to SDWE
This implementation belongs to RDWS
This implementation belongs to SDWS


### Qiskit

Next, we will consider implementations from Qiskit. In Qiskit, MCT can be implemented with parameters v-chain, v-chain-dirty, and recursion, which we wrapped inside the functions MCTVChain, MCTVChainDirty, and MCTRecursion. 

In [8]:
control_no = 5
mct = MCTVChain(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation belongs to SCNW
This implementation belongs to RCNW
This implementation belongs to SCWE
This implementation belongs to RCWS
This implementation belongs to SCWS
This implementation doesn't belong to SDNW
This implementation doesn't belong to RDNW
This implementation doesn't belong to SDWE
This implementation doesn't belong to RDWS
This implementation doesn't belong to SDWS


In [9]:
control_no = 5
mct = MCTVChainDirty(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation belongs to SCNW
This implementation belongs to RCNW
This implementation belongs to SCWE
This implementation belongs to RCWS
This implementation belongs to SCWS
This implementation belongs to SDNW
This implementation belongs to RDNW
This implementation belongs to SDWE
This implementation belongs to RDWS
This implementation belongs to SDWS


Since V-Chain-Dirty belongs to "Strict Dirty Non-Wasting" class, it passes all the verifications.

In [10]:
control_no = 5
mct = MCTRecursion(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation belongs to SCNW
This implementation belongs to RCNW
This implementation belongs to SCWE
This implementation belongs to RCWS
This implementation belongs to SCWS
This implementation belongs to SDNW
This implementation belongs to RDNW
This implementation belongs to SDWE
This implementation belongs to RDWS
This implementation belongs to SDWS


Like V-Chain-Dirty, recursion belongs to "Strict Dirty Non-Wasting" class and it passes all the verifications.

### Parallel Decomposition

This is the implementation based on the logarithmic depth construction from the paper 'Efficient Constructions for Simulating Multi Controlled Quantum Gates' https://dx.doi.org/10.1007/978-3-031-08760-8_16 
The Toffoli gates used here are strict. The paper uses relative Toffoli gates.

In [11]:
control_no = 5
mct = MCTParallelDecomposition(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)


This implementation belongs to SCNW
This implementation belongs to RCNW
This implementation belongs to SCWE
This implementation belongs to RCWS
This implementation belongs to SCWS
This implementation doesn't belong to SDNW
This implementation doesn't belong to RDNW
This implementation doesn't belong to SDWE
This implementation doesn't belong to RDWS
This implementation doesn't belong to SDWS


### QCLIB - LDMCU

This is the no-auxilliary implementation from https://arxiv.org/abs/2203.11882

In [12]:
control_no = 5
mct = MCTQclibLdmcu(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation belongs to SCNW
This implementation belongs to RCNW
This implementation belongs to SCWE
This implementation belongs to RCWS
This implementation belongs to SCWS
This implementation belongs to SDNW
This implementation belongs to RDNW
This implementation belongs to SDWE
This implementation belongs to RDWS
This implementation belongs to SDWS


## Clean and Dirty Wasting Entangling implementations

Finally, below we provide an implementations designed in [here](...), which are the only members of the Strict and Dirty Wasting Entangling classes intruduce in the same paper.

In [13]:
control_no = 4
mct = MCTCleanWastingEntangling(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation doesn't belong to SCNW
This implementation doesn't belong to RCNW
This implementation belongs to SCWE
This implementation doesn't belong to RCWS
This implementation doesn't belong to SCWS
This implementation doesn't belong to SDNW
This implementation doesn't belong to RDNW
This implementation doesn't belong to SDWE
This implementation doesn't belong to RDWS
This implementation doesn't belong to SDWS


In [7]:
control_no = 5
mct = MCTDirtyWastingEntangling(control_no)
circ = transpile(mct.generate_circuit(), basis_gates=["cx", "u3"])
tested_matrix = usim.run(circ).result().get_unitary().data
ref_unitary = get_ref_unitary(control_no)
verify_all(tested_matrix, ref_unitary)

This implementation doesn't belong to SCNW
This implementation doesn't belong to RCNW
This implementation belongs to SCWE
This implementation doesn't belong to RCWS
This implementation doesn't belong to SCWS
This implementation doesn't belong to SDNW
This implementation doesn't belong to RDNW
This implementation belongs to SDWE
This implementation doesn't belong to RDWS
This implementation doesn't belong to SDWS


# Transformations

Now we'll look at Lemma 7.2 from Barenco et al. and use it to showcase some of the transformations from the paper.

Below we define a function to implement Lemma 7.2

In [8]:
import numpy as np
from qiskit import QuantumCircuit, Aer

controls_to_check = 5


def lemma_7_2(num_controls=5):
    if num_controls < 3:
        raise ValueError("Number of controls must be >=3")

    n = 2 * num_controls - 1
    assert np.ceil(n / 2) == num_controls

    qc = QuantumCircuit(n)

    auxs = list(range(num_controls, n - 1))
    num_aux = len(auxs)

    controls = list(range(num_controls))
    target = n - 1
    print(n, controls, auxs, target)

    for i, c2 in enumerate(auxs[::-1]):
        qc.ccx(c2 - num_aux, c2, target - i)
    qc.ccx(0, 1, auxs[0])
    for i, c2 in enumerate(auxs):
        qc.ccx(c2 - num_aux, c2, target - num_aux + 1 + i)

    for i, c2 in enumerate(auxs[::-1]):
        if c2 == auxs[-1]:
            continue
        qc.ccx(c2 - num_aux, c2, target - i)
    qc.ccx(0, 1, auxs[0])
    for i, c2 in enumerate(auxs):
        if c2 == auxs[-1]:
            continue
        qc.ccx(c2 - num_aux, c2, target - num_aux + 1 + i)

    MCT = QuantumCircuit(n)
    MCT.mct(controls, target, auxs)

    return qc, MCT, controls, auxs, target

In [9]:
simulator = Aer.get_backend('aer_simulator')
circ, mct, controls, auxs, target = lemma_7_2(controls_to_check)
print(circ.draw(fold=-1))

9 [0, 1, 2, 3, 4] [5, 6, 7] 8
                                                                 
q_0: ─────────────────■─────────────────────────────■────────────
                      │                             │            
q_1: ─────────────────■─────────────────────────────■────────────
                      │                             │            
q_2: ────────────■────┼────■───────────────────■────┼────■───────
                 │    │    │                   │    │    │       
q_3: ───────■────┼────┼────┼────■─────────■────┼────┼────┼────■──
            │    │    │    │    │         │    │    │    │    │  
q_4: ──■────┼────┼────┼────┼────┼────■────┼────┼────┼────┼────┼──
       │    │    │  ┌─┴─┐  │    │    │    │    │  ┌─┴─┐  │    │  
q_5: ──┼────┼────■──┤ X ├──■────┼────┼────┼────■──┤ X ├──■────┼──
       │    │  ┌─┴─┐└───┘┌─┴─┐  │    │    │  ┌─┴─┐└───┘┌─┴─┐  │  
q_6: ──┼────■──┤ X ├─────┤ X ├──■────┼────■──┤ X ├─────┤ X ├──■──
       │  ┌─┴─┐└───┘     └───┘┌─┴─┐  │  ┌─┴─┐└

/var/folders/v6/623nkjns6rs5jqzt737r741m0000gn/T/ipykernel_64540/4061057848.py:40: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.mct()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use QuantumCircuit.mcx as direct replacement.
  MCT.mct(controls, target, auxs)


This circuit assumes dirty auxiliary qubits. If we have clean auxiliaries, and the implementation is non-wasted, the circuit can be transformed to reduce the number of gates used.

Below, we have a function which takes a circuit and it's control and auxiliary qubits and outputs a transformed clean circuit.
(Note: This function is an example and does not work for a general circuit.)

In [10]:
def transform_clean_nonwasted(circuit, all_controls, all_auxs, target):
    all_auxs_copy = all_auxs.copy()
    num_all_controls = len(all_controls)
    num_all_auxs = len(all_auxs)

    instructions = circuit.data
    print(len(instructions))
    remove_ins = []

    for i, instruction in reversed(list(enumerate(instructions))):
        c1 = instruction.qubits[0]._index
        c2 = instruction.qubits[1]._index
        t = instruction.qubits[2]._index
        if c1 in all_auxs_copy:
            remove_ins.append(i)
            continue
        if c2 in all_auxs_copy:
            remove_ins.append(i)
            continue
        if t in all_auxs_copy:
            all_auxs_copy.remove(t)
            continue

    all_auxs_copy = all_auxs.copy()
    for i, instruction in enumerate(instructions):
        c1 = instruction.qubits[0]._index
        c2 = instruction.qubits[1]._index
        t = instruction.qubits[2]._index
        if c1 in all_auxs_copy:
            remove_ins.append(i)
            continue
        if c2 in all_auxs_copy:
            remove_ins.append(i)
            continue
        if t in all_auxs_copy:
            all_auxs_copy.remove(t)
            continue

    print(remove_ins)
    remaining_instructions = [i for j, i in enumerate(instructions) if j not in set(remove_ins)]

    clean_circuit = QuantumCircuit(num_all_controls + num_all_auxs + 1)
    for instruction in remaining_instructions:
        c1 = instruction.qubits[0]._index
        c2 = instruction.qubits[1]._index
        t = instruction.qubits[2]._index
        clean_circuit.ccx(c1, c2, t)

    return clean_circuit

In [11]:
print(controls, auxs, target)
clean_circ = transform_clean_nonwasted(circ, controls, auxs, target)
print(clean_circ.draw(fold=-1))

[0, 1, 2, 3, 4] [5, 6, 7] 8
12
[11, 10, 0, 1, 2]
                                        
q_0: ──■─────────────────────────────■──
       │                             │  
q_1: ──■─────────────────────────────■──
       │                             │  
q_2: ──┼────■───────────────────■────┼──
       │    │                   │    │  
q_3: ──┼────┼────■─────────■────┼────┼──
       │    │    │         │    │    │  
q_4: ──┼────┼────┼────■────┼────┼────┼──
     ┌─┴─┐  │    │    │    │    │  ┌─┴─┐
q_5: ┤ X ├──■────┼────┼────┼────■──┤ X ├
     └───┘┌─┴─┐  │    │    │  ┌─┴─┐└───┘
q_6: ─────┤ X ├──■────┼────■──┤ X ├─────
          └───┘┌─┴─┐  │  ┌─┴─┐└───┘     
q_7: ──────────┤ X ├──■──┤ X ├──────────
               └───┘┌─┴─┐└───┘          
q_8: ───────────────┤ X ├───────────────
                    └───┘               


Here, we see that the transformed circuit has removed gates that would have had no action on clean auxiliaries.

Next, we have a function that can take a circuit and transform it into its wasted version.
(Note: This function is an example and does not work for a general circuit.)

In [12]:
def transform_to_wasting(circuit, all_controls, all_auxs, target):
    all_auxs = all_auxs.copy()
    num_all_controls = len(all_controls)
    num_all_auxs = len(all_auxs)

    instructions = circuit.data
    remove_ins = []
    for i, instruction in enumerate(instructions[::-1]):
        c1 = instruction.qubits[0]._index
        c2 = instruction.qubits[1]._index
        t = instruction.qubits[2]._index
        if t in all_auxs:
            remove_ins.append(i)
            continue
        else:
            break

    remaining_instructions = [i for j, i in enumerate(instructions[::-1]) if j not in set(remove_ins)]

    wasted_circuit = QuantumCircuit(num_all_controls + num_all_auxs + 1)
    for instruction in remaining_instructions[::-1]:
        if instruction.operation.name == 'ccx':
            c1 = instruction.qubits[0]._index
            c2 = instruction.qubits[1]._index
            t = instruction.qubits[2]._index
            wasted_circuit.ccx(c1, c2, t)

    return wasted_circuit

In [13]:
wasted_circ = transform_to_wasting(circ, controls, auxs, target)
print(wasted_circ.draw(fold=-1))

                                        
q_0: ─────────────────■─────────────────
                      │                 
q_1: ─────────────────■─────────────────
                      │                 
q_2: ────────────■────┼────■────────────
                 │    │    │            
q_3: ───────■────┼────┼────┼────■───────
            │    │    │    │    │       
q_4: ──■────┼────┼────┼────┼────┼────■──
       │    │    │  ┌─┴─┐  │    │    │  
q_5: ──┼────┼────■──┤ X ├──■────┼────┼──
       │    │  ┌─┴─┐└───┘┌─┴─┐  │    │  
q_6: ──┼────■──┤ X ├─────┤ X ├──■────┼──
       │  ┌─┴─┐└───┘     └───┘┌─┴─┐  │  
q_7: ──■──┤ X ├───────────────┤ X ├──■──
     ┌─┴─┐└───┘               └───┘┌─┴─┐
q_8: ┤ X ├─────────────────────────┤ X ├
     └───┘                         └───┘


We can see that removing the gates that act to get the auxiliaries back to their original state, we can reduce the number of gates significantly but at the cost of having wasted auxiliaries.

We can use the clean auxiliary circuit from before and pass it through this function to get a clean wasted circuit.

In [14]:
clean_wasted_circ = transform_to_wasting(clean_circ, controls, auxs, target)
print(clean_wasted_circ.draw(fold=-1))

                         
q_0: ──■─────────────────
       │                 
q_1: ──■─────────────────
       │                 
q_2: ──┼────■────────────
       │    │            
q_3: ──┼────┼────■───────
       │    │    │       
q_4: ──┼────┼────┼────■──
     ┌─┴─┐  │    │    │  
q_5: ┤ X ├──■────┼────┼──
     └───┘┌─┴─┐  │    │  
q_6: ─────┤ X ├──■────┼──
          └───┘┌─┴─┐  │  
q_7: ──────────┤ X ├──■──
               └───┘┌─┴─┐
q_8: ───────────────┤ X ├
                    └───┘
